```python
df.groupby('QLabel').filter(lambda x: x['year'].nunique() <= 3)
```

In [1]:
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

d:\Software\Anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
df=pd.read_csv("r1.csv")

In [2]:
CPCleanedUnique=df.CPCleaned.unique()

def collapse(cp):
    for i in df[df.CPCleaned==cp].QCleaned.unique():
        print(i,"\n",tabCountry(i),"\n---------------------------\n", getQNum(i),sep="")
        
def getOriginalQ(cleanedQ):
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

with open("grpdQ.txt","r") as txt:
    grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

In [172]:
def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def assign(q,cpname):
    global df
    df.loc[df.QCleaned==q,"CPCleaned"]=cpname

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def dropdel():
    df.drop(df[df.CPCleaned=="del"].index,inplace=True)

def tab(cp):
    global df
    filtered_df=df[df['CPCleaned'] ==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def getcp(q):
    global df
    return list(df[df.QCleaned==q].CPCleaned.unique())

In [3]:
df.cleaned.value_counts()

0    820466
1    233138
Name: cleaned, dtype: int64

---

In [104]:
multicpidx=[]
for idx in range(len(grpdQ)):
    i=grpdQ[idx]
    if len(i)>1:
        cpl=[]
        for x in i:
            cpl.extend(getcp(x))
        cpset=set(cpl)

        if len(cpset)>1:
            multicpidx.append(idx)

In [112]:
with open("grpdRes.txt", "r") as gR:
    grl=gR.read().split("\n")[:-2]

for x in grl:
    idx= int(x.split(":")[0].split(" ")[-1])
    if idx not in multicpidx and int(x.split(":")[-1])<=6476:
        replace(grpdQ[idx])

----

In [114]:
replace(["Trained workers on fire fighting","Has the employer appointed and trained a fire management team?"])

In [121]:
for q in ["Escape routes locked",
"Has the employer elaborated and implemented an emergency plan?",
"Does the employer comply with size requirements for escape routes?",
"Does the workplace have adequate stairways that can be used in case of emergency?",
"Does the workplace have at least one adequate window exit per floor?",
"Are floors and corridors in line with legal requirements?",
"Are the emergency exits  unlocked during working hours, including overtime?"]:
    discard(q)

# #droped:
# Escape routes locked                                                                 4994
# Has the employer elaborated and implemented an emergency plan?                       1921
# Does the employer comply with size requirements for escape routes?                   1802
# Does the workplace have adequate stairways that can be used in case of emergency?     584
# Does the workplace have at least one adequate window exit per floor?                  566
# Are floors and corridors in line with legal requirements?                             117
# Are the emergency exits  unlocked during working hours, including overtime?            12

In [122]:
CPVC(0)

Emergency Preparedness


Accessable unlocked emergency exits                            6476
Fire detection and alarm system                                6476
At least 2 possible emergency exits                            6476
Periodic emergency drills                                      6476
Does the workplace have adequate fire-fighting equipment?      6476
Trained workers on fire fighting                               6470
Emergency exits/escape routes clearly marked                   6269
Flammable materials safely stored                              6225
Are possible sources of ignition appropriately safeguarded?    6212
Name: QCleaned, dtype: int64

# 0: cleaned
----

In [127]:
CPVC(1)

Overtime Wages


Payment of overtime                                                                                                          6476
Does the employer pay all workers the correct meal allowance or give them a reasonable free meal when they work overtime?    2503
Does the employer pay workers double their normal wage for all hours worked on weekly rest days?                             2503
Does the employer pay workers double their normal wage for all overtime hours worked at night?                               2238
Does the employer pay workers correctly for all overtime hours worked at night?                                              1919
Does the employer pay workers correctly for all overtime hours worked on public holidays?                                    1802
Does the employer pay workers correctly for all overtime hours worked on weekly rest days?                                   1802
Does the employer provide meals and drinks of at least 1,400 calories to workers working o

In [134]:
# replace(["Does the employer pay workers double their normal wage for all hours worked on weekly rest days?","pay correctly on weekly rest day?"
#          ,"Does the employer pay workers 150% of their normal wage for overtime worked on weekly rest days and public holidays?","Does the employer pay workers 100% more than their normal wage for all overtime hours worked on weekly rest days?",
#          "Does the employer pay workers correctly for all overtime hours worked on weekly rest days?"])

# replace(["Does the employer provide meals and drinks of at least 1,400 calories to workers working overtime for 3 hours or more?"
#         ,"Does the employer pay all workers the correct meal allowance or give them a reasonable free meal when they work overtime?"])

# replace(["Does the employer pay workers correctly for all overtime hours worked at night?"
#         ,"Does the employer pay workers double their normal wage for all overtime hours worked at night?"])
tab("Overtime Wages")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Does the employer pay piece rate workers correctly for all overtime hours worked on regular work days?,584,0,0,0,0,0,0,584
Does the employer pay workers correctly for all overtime hours worked at night?,0,2238,0,0,0,117,1802,4157
Does the employer pay workers correctly for all overtime hours worked on public holidays?,0,0,0,0,0,0,1802,1802
"Does the employer provide meals and drinks of at least 1,400 calories to workers working overtime for 3 hours or more?",0,2503,0,833,0,0,0,3336
Payment of overtime,584,2503,161,833,474,119,1802,6476
pay correctly on weekly rest day?,584,2503,0,0,474,119,1802,5482
All,1752,9747,161,1666,948,355,7208,21837


Does the employer provide meals and drinks of at least 1,400 calories to workers working overtime for 3 hours or more?	,Indonesia

# 1:cleaned
---
# 2,3:skipped
---
# 4: cleaned
---

In [164]:
tab("Chemicals and Hazardous Substances")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Are chemical mixing rooms properly equipped and used?,0,2238,0,0,0,0,0,2238
Chemicals/hazardous substance properly labelled,584,2503,156,833,474,119,1802,6471
Chemicals/hazardous substance properly stored,584,2293,161,833,0,119,1802,5792
Does the employer keep an inventory of hazardous chemicals used in the workplace?,0,0,0,0,0,0,1802,1802
Does the employer keep chemical safety records for the hazardous chemicals used in the workplace?,584,2279,161,0,0,119,1802,4945
Does the employer provide information on the chemicals and hazardous substances used in the workplace to the Ministry of Labour?,0,0,0,0,0,119,0,119
Has the employer appointed a Chemical Officer?,0,0,0,833,0,0,0,833
"Has the employer prepared for response, control and cleaning of chemical spills?",0,2270,0,0,0,0,0,2270
Storage of chemicals and hazardous substances,584,2503,161,833,474,119,1802,6476


In [ ]:
#*
undo('Storage of chemicals and hazardous substances')

In [163]:
# replace([
#     "Has the employer taken action to prevent and limit workers' exposure to chemicals and hazardous substances?",
#     "Has the employer taken action to assess, monitor, prevent and limit workers' exposure to chemicals and hazardous substances?",
#     "Has the employer taken action to, prevent and limit workers' exposure to chemicals and hazardous substances?",
# ])

# replace([
# "Has the employer taken action to  prevent and limit workers' exposure to chemicals and hazardous substances?",
# "Has the employer taken action to prevent and limit workers' exposure to chemicals and hazardous substances?"
# ],"prevent workers' exposure to hazardous substances?")

# replace([
#     "Does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace, and are the chemicals properly labelled?",
#     "Chemicals/hazardous substance properly labelled",
#     "Does the employer keep a chemical inventory, and ensure that all chemicals and hazardous substances are properly labelled and have safety data sheets?"
# ])

# replace([
#     "Chemicals/hazardous substance properly stored"
#     ,"Does the employer keep an inventory of chemicals and hazardous substances used in the workplace?"
# ])

# replace(["Does the employer provide adequate washing facilities and cleaning materials in the event of exposure to hazardous chemicals?",
#         "Does the employer provide adequate washing facilities and cleansing materials in the event of exposure to hazardous chemicals?"],
#         "adequate washing facilities for exposure")

# replace(["Has the employer effectively trained workers who work with chemicals and hazardous substances?",
#          "Has the employer trained all workers who work with chemicals and hazardous substances?",
#          "Has the employer effectively trained workers and supervisors who work with or are responsible for hazardous chemicals?",
#          "Has the employer effectively trained workers and supervisor who work with or are responsible for hazardous chemicals?"],"Training on hazardous chemicals")

# replace(["Does the employer have chemical safety data sheets for all the hazardous chemicals used in the workplace?",
#         "Does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace?"])

# replace(["Does the employer keep chemical safety records for the hazardous chemicals used in the workplace?",
#          "Does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace?"])

# *5: cleaned
---
# 6: cleaned
---

In [189]:
tab("Paid Leave")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Do pregnant workers receive correct payment when they take time off for prenatal care?,0,0,0,0,0,0,1802,1802
Does the employer pay for paternity leave when required?,0,0,0,0,0,0,1802,1802
Does the employer pay workers correctly during casual leave?,584,0,0,0,0,0,0,584
Does the employer pay workers correctly during special leave?,0,2503,0,0,0,0,0,2503
Does the employer pay workers correctly during work stoppages?,584,2503,0,833,474,0,0,4394
Does the employer pay workers correctly for paid public holidays?,584,2243,133,833,474,0,1802,6069
Does the employer pay workers correctly for personal leave?,0,0,0,0,0,0,1802,1802
Does the employer pay workers correctly when they take time off for paternity leave?,0,0,0,0,0,117,0,117
"Does the employer pay workers correctly when they take time off for personal/ family matters, as required by national law?",0,0,0,833,0,0,0,833


In [190]:
# replace([
#     "Does the employer pay workers correctly for festival holidays (including compensatory festival holidays when workers work on festival holidays)?",
#     "Does the employer pay workers correctly for paid public holidays?",
#     "Does the employer pay workers correctly for legally mandated holidays?	",
#     "Does the employer pay workers for legally mandated paid public holidays?"
# ])

# replace([
#     "Does the employer pay workers correctly during annual leave?",
#     "Paid annual leave","Does the employer pay workers correctly for legally required annual leave?"
# ])

# replace([
#     "Paid sick leave",
#     "Does the employer provide paid sick leave as required by the factory’s Internal Regulations, or if there are no Internal Regulations, according to the Ministry's practice?",
#     "Do workers receive correct payment during sick leave and other types of leave that are covered by social insurance?"
# ])


# replace(["Does the employer pay workers correctly during suspensions of work?",
#          "Does the employer pay workers correctly during work stoppages?"
#          ])

# replace(["Paid leave for breastfeed	","Does the employer pay workers correctly for the one hour of paid time off per day for breastfeeding?",
#          "Does the employer provide workers one hour of paid time off per day for breast-feeding?","Does the employer pay workers for one hour breastfeeding break per day?"])


# replace(["Does the employer pay women workers for 30 minutes rest per day during their periods?","Does the employer pay workers correctly when they feel pain during the first two days of menstruation and they notify the employer?",
#          "Paid rest and breaks"],
#         "pay for menstruation rest")


# *7: cleaned
---
# 8: cleaned (lack Indonesia only)
---

In [196]:
tab("Interference and Discrimination")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Are workers free to meet without management present?,584,2237,161,833,474,119,1802,6210
"Does the employer threaten, intimidate, or harass workers who join a union or engage in union activities?",584,2503,161,833,474,119,1802,6476
Has the employer terminated a union official without getting permission from the Ministry?,0,2503,0,0,0,0,0,2503
Has the employer terminated a union official without the written agreement of the union board or the higher-level union?,0,0,0,0,0,0,1775,1775
"If there is more than one union, does the employer treat them equally?",584,1350,161,833,474,119,27,3548
Interfere union,566,2503,161,833,474,119,1802,6458
Is a job applicant's union membership or union activities a factor in hiring decisions?,584,2503,161,833,474,119,1802,6476
"Is the employer involved in union decision making, the formation of the constitution and rules, in union activities, administration, finances or elections?",0,1350,0,0,0,0,0,1350
Punish unionists,584,2503,161,833,474,119,1802,6476


In [195]:
# replace([
#     "Does the employer provide incentives to workers to keep them from joining a union or engaging in union activities?",
#     "Has the employer tried to interfere with, manipulate, or control the union(s)?"
# ],"intention to interfere union")

# *9: cleaned
---

In [273]:
tab("Dialogue, Discipline and Disputes")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Bullying harassment,584,2503,161,833,474,119,1802,6476
Did the employer resolve grievances and disputes in compliance with legal requirements?,584,2503,161,833,474,0,1802,6357
Do disciplinary measures comply with legal requirements?,584,2236,0,833,474,119,1802,6048
Does the factory have a functioning Participation Committee?,584,2411,0,833,474,0,1802,6104
Has the employer complied with arbitration awards?,0,2503,0,0,0,0,0,2503
Has the employer complied with court orders arising from individual or collective disputes?,0,2237,0,0,0,0,0,2237
Has the employer complied with final conciliation agreements?,0,2503,0,0,0,0,0,2503
Qualified welfare officer,584,0,0,0,0,0,0,584
All,2920,16896,322,3332,1896,238,7208,32812


In [269]:
def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

In [200]:
# replace([
#     "Do disciplinary measures comply with legal requirements?",
#     "Do the disciplinary measures comply with the company's by-laws approved by the Ministry of Labor?	",
#     "Are disciplinary measures proportional to the seriousness of workers' misconduct?	"
# ])

# replace(["Does the factory have a functioning Participation Committee?",
#          "Does dialogue at the workplace take place in compliance with legal requirement?"])

# *10: cleaned
---

In [204]:
tab("Coercion")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"Are workers free to terminate their employment with reasonable notice, and to leave their jobs when their contracts expire?",566,2238,156,0,474,0,1496,4930
Does the employer delay or withhold wage payments in order to coerce workers to work?,584,2238,161,833,474,0,1802,6092
"Does the employer deny workers access to their personal documents (such as birth certificates, passports, work permits and ID cards) when they need them?",584,0,161,833,0,0,1802,3380
Does the employer force workers to work to discipline them or as punishment for participation in a strike?,584,2236,161,833,474,0,1802,6090
Does the employer restrict workers' freedom to come and go from the dormitories and/or the industrial park or zone in which the factory is located?,584,2503,161,833,474,119,1802,6476
"Does the employer use threats such as deportation, cancellation of visas or reporting to the authorities in order to force workers to stay at the job?",584,0,161,833,474,119,1802,3973
Does the employer use violence or the threat of violence to intimidate workers?,584,2236,161,833,474,119,1802,6209
Free exit from workplace at all times,584,2503,161,833,474,119,1802,6476
All,4654,13954,1283,5831,3318,476,14110,43626


# *11: cleaned
---
# 12: cleaned
---

In [227]:
tab("OSH Management Systems")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Adequate OSH policy,0,2412,0,0,0,0,0,2412
Does the employer conduct risk assessment?,584,2001,161,833,474,119,1802,5974
Does the employer develop an OSH plan annually?,0,0,0,0,476,0,1802,2278
Does the employer have an OSH license?,0,0,0,0,0,119,0,119
Does the employer have the required certificates for the installation/ operation of electrical installations and other machines and equipment?,0,0,0,833,0,0,0,833
Does the employer inspect and measure the environmental conditions in the workplace on a annual basis?,0,0,0,0,0,0,1802,1802
"Does the employer record and investigate work-related accidents and illnesses, and indicate the technical recommendations necessary to prevent them?",0,0,0,0,0,117,0,117
Does the employer record work-related accidents and diseases?,584,2499,161,833,474,117,1802,6470
"Does the employer regularly inspect and maintain machines, equipment, buildings and stores?",0,0,0,0,0,0,1802,1802


In [226]:
# replace(["Has legal construction permits/structural safety certificate","Does the employer have legally required construction/building permits?	",
#          ])

replace(["Does the employer record work-related accidents and diseases?	","Does the employer record work-related accidents and diseases and submit the record to OFATMA on a monthly basis?",
         "Does the employer record work-related accidents and illnesses?","Does the employer keep updated records of work-related accidents and diseases?	"
         ,"Does the employer record work-related accidents and illnesses and report them to the Ministry?",
         "Does the employer record work-related accidents and diseases and inform the governmental authorities as required by law?	",
         "Does the employer record and report work-related accidents and diseases to the Local Manpower Office?",
         "Does the employer record, investigate and report work-related accidents and diseases to the Local Manpower Office?"])

# replace(["Has the employer performed initial and regular reviews of occupational safety and health issues in the factory?	",
#          "Has the employer performed an assessment of general occupational safety and health issues in the factory?",
#          "Has the employer done an initial industrial hygiene risk assessment, a risk map, and annual risk assessments?	",
#          "Does the employer conduct risk assessment?","Has the employer performed hazard/risk management and control and conducted regular reviews of occupational safety and health performance in the factory?"],)

# 13: ?cleaned
---

In [292]:
tab("Minimum Wages/Piece Rate Wages")

ValueError: No objects to concatenate

In [290]:
df.drop(df.QCleaned.isin(["Does the employer pay apprentices the wage agreed to by the parties?",
                      "Does the employer pay at least 85% of the wage paid to other workers for the same job for ordinary hours of work to probationary workers?",
                      "Is the piece rate set at a level that permits workers of average ability working normal hours to earn minimum wage?"]).index,inplace=True)

In [236]:
# replace([
#     "Are piece-rate workers paid correctly for ordinary hours of work?",
# "Paid piece rate workers correctluy"
#     ],"Paid piece rate workers correctly")

replace([
    "Does the employer pay higher than district minimum wage for ordinary hours of work to permanent (PKWTT) and non-permanent workers (PKWT) who have worked for more than 1 year?",
    "Does the employer pay at least minimum wage for temporary and probationary workers?",
    "Does the employer pay at least 7% higher than the applicable legal minimum wage for ordinary hours of work to workers who have received vocational training?"
],"minimum wage for non-permanent workers")

# 14: *cleaned
---

In [238]:
CPVC(15)

Strikes


Punish workers for striking                                                                                                      6476
Has the employer hired new workers to replace striking workers during a strike?                                                  6209
Has the employer tried to prevent any workers from participating in a strike?                                                    4407
Has the employer called security guards, the police or armed forces to break up a peaceful strike or arrest striking workers?    1802
Has the employer tried to prevent workers from participating in a strike?                                                        1802
Name: QCleaned, dtype: int64

In [239]:
tab("Strikes")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"Has the employer called security guards, the police or armed forces to break up a peaceful strike or arrest striking workers?",0,0,0,0,0,0,1802,1802
Has the employer hired new workers to replace striking workers during a strike?,584,2236,161,833,474,119,1802,6209
Has the employer tried to prevent any workers from participating in a strike?,584,2236,161,833,474,119,0,4407
Has the employer tried to prevent workers from participating in a strike?,0,0,0,0,0,0,1802,1802
Punish workers for striking,584,2503,161,833,474,119,1802,6476
All,1752,6975,483,2499,1422,357,7208,20696


In [241]:
df[df.QCleaned=="Has the employer tried to prevent workers from participating in a strike?"].AssesmentDate.str[-4:].unique()

array(['2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype=object)

In [242]:
df[df.QCleaned=="Has the employer called security guards, the police or armed forces to break up a peaceful strike or arrest striking workers?"].AssesmentDate.str[-4:].unique()

array(['2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype=object)

In [243]:
df.to_csv("r2.csv", index=False)